## 기본 라이브러리 불러오기

In [2]:
# 라이브러리 임포트
import os
import numpy as np
import pandas as pd
import chardet

# 사용자 정의함수 경로 설정
import sys
sys.path.append("G:/내 드라이브/Source/_Custom_Function")
# 사용자 정의함수 불러오기

pd.set_option('display.max_column', None)

import time

import ExcelFile_Merge as em
import ExcelFile_Read as er
import ExcelFile_EDA as ee

---
### 이상치 제거 함수

In [4]:
def check_outlier(_df, _column):
    # IQR 방법
    Q1 = _df[_column].quantile(0.25)
    Q3 = _df[_column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = _df[~((_df[_column] < lower_bound) | (_df[_column] > upper_bound))]
    
    return outliers

---
## 이상치패턴 저장데이터 다시 불러오기

In [6]:
start_time = time.time()
df_outlier_rental = pd.read_parquet('G:\\내 드라이브\\DataSet\\_파킷 파일\\서울특별시 공공자전거 대여이력 정보\\서울특별시 공공자전거 대여이력 정보_(PRE)고장예측_2023_2024_20241107.parquet')
print("read_csv time :", time.time() - start_time)

read_csv time : 0.05644369125366211


In [7]:
df_outlier_rental.head()

,자전거번호,이전일자_1mon,등록대여일자,고장구분,고장여부,이상패턴_5min_횟수,이전일자_1mon_이용거리,이전일자_1mon_이용시간,이전일자_1mon_이용횟수
0,SPB-30006,2024-02-29,2024-03-31,페달,1,2,98170.92,843.0,74.0
1,SPB-30010,2024-05-18,2024-06-18,타이어,1,5,231653.56,2257.0,121.0
2,SPB-30014,2024-05-24,2024-06-24,기타,1,7,335138.54,2966.0,156.0
3,SPB-30018,2024-05-01,2024-06-01,기타,1,2,291259.65,3041.0,153.0
4,SPB-30022,2024-04-20,2024-05-20,기타,1,7,253399.66,2755.0,143.0


In [8]:
df_outlier_rental['고장여부'].value_counts()

고장여부
1    28589
0    13307
Name: count, dtype: int64

---
## 23년 자전거대여 월별집계 내역 불러오기

In [10]:
start_time = time.time()
df_2023_rental = pd.read_parquet('G:\\내 드라이브\\DataSet\\_파킷 파일\\서울특별시 공공자전거 대여이력 정보\\서울특별시 공공자전거 대여이력 정보_(PRE)2023_자전거별_월별_집계.parquet')
print("read_csv time :", time.time() - start_time)

FileNotFoundError: [Errno 2] No such file or directory: 'G:\\내 드라이브\\DataSet\\_파킷 파일\\서울특별시 공공자전거 대여이력 정보\\서울특별시 공공자전거 대여이력 정보_(PRE)2023_자전거별_월별_집계.parquet'

In [ ]:
df_2023_rental.shape[0]

In [ ]:
df_2023_rental.head(5)

In [ ]:
df_2023_rental.columns

In [ ]:
df_2023_rental = df_2023_rental[['자전거번호', '대여일자_count(mean)', '이용거리(M)_mean(mean)', '이용거리(M)_sum(mean)']]

In [ ]:
df_outlier_rental = pd.merge(df_outlier_rental, df_2023_rental, on='자전거번호', how='inner')

In [ ]:
df_outlier_rental.head()

In [ ]:
df_outlier_rental.isnull().sum()

---
### 23년 월별평균 <-> 한달 이전데이터 비교 컬럼 추가

In [ ]:
# 고장 자전거는 23년 월별평균보다, 이전 한달 데이터가 더 적을 것이라는 가정
df_outlier_rental['이용횟수_비교'] = ee.ss_compare(df_outlier_rental['대여일자_count(mean)'], df_outlier_rental['이전일자_1mon_이용횟수'])
df_outlier_rental['이용거리_비교'] = ee.ss_compare(df_outlier_rental['이용거리(M)_sum(mean)'], df_outlier_rental['이전일자_1mon_이용거리'])

In [ ]:
df_outlier_rental['이용횟수_비교(절대값)'] = df_outlier_rental['이용횟수_비교'].abs()
df_outlier_rental['이용거리_비교(절대값)'] = df_outlier_rental['이용거리_비교'].abs()

In [ ]:
df_outlier_rental.head()

In [ ]:
df_Bike_latest_2 = df_outlier_rental

In [ ]:
df_Bike_latest_2.head()

---
## 고장횟수 별 - 고장비율
 - 고장 : 24년의 마지막고장신고
 - 정상 : 24년의 마지막 대여일

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

### [전체] 고장여부 비율

In [ ]:
counts = df_Bike_latest_2['고장여부'].value_counts()
total = counts.sum()   # 총 데이터 개수
percentage = counts / total * 100   # 비율 계산
print(f'total : {total}')
print(f'counts : {counts}')
print(f'percentage : {percentage}')

---
## 고장여부 에 따른 고장횟수

In [ ]:
# 한글 폰트 설정 (Windows에서는 'Malgun Gothic' 사용)
plt.rcParams['font.family'] = 'Malgun Gothic'
# 마이너스 기호 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False

# 데이터 준비
gojang_counts = df_Bike_latest_2['고장여부'].value_counts()
# 그래프 크기 설정
plt.figure(figsize=(10, 6))
# 막대 그래프 그리기
gojang_counts.plot(kind='bar', color=['skyblue', 'lightcoral'])
# 그래프 꾸미기
plt.title('고장 여부 분포', fontsize=16)
plt.xlabel('고장 여부', fontsize=12)
plt.ylabel('빈도수', fontsize=12)
plt.xticks(rotation=0)

# 각 막대 위에 값 표시
for i, v in enumerate(gojang_counts):
    plt.text(i, v, str(v), ha='center', va='bottom')

plt.tight_layout()
plt.show()

### 결측치 확인

In [ ]:
df_Bike_latest_2.isnull().sum()

In [ ]:
import missingno as msno
# 결측치 매트릭스 시각화
msno.matrix(df_Bike_latest_2)
plt.show()

---
### 중복제거

In [ ]:
df_Bike_latest_3 = df_Bike_latest_2.dropna()

---
### subplot multi 함수선언

In [ ]:
# 서브플롯 생성 함수
def create_bi_subplot(_df, _columns):
    plt.figure(figsize=(15, 6))
    
    # 전체 데이터에서 x축 범위 계산
    # x_min = min(_df[_columns[0]].min(),
    #             _df[_columns[1]].min(),
    #             _df[_columns[2]].min())
    # x_max = max(_df[_columns[0]].max(),
    #             _df[_columns[1]].max(),
    #             _df[_columns[2]].max())
    x_min = _df[_columns[0]].min()
    x_max = _df[_columns[0]].max()

    # 최대 빈도수 계산
    max_count = 0
    for j in [0, 1]:
        for _column in _columns:
            data = _df[_df['고장여부'] == j][_column]
            counts, _ = np.histogram(data, bins=10)
            max_count = max(max_count, counts.max())

    
    position = 1
    for j in list(range(0, 2)):
        for i, _column in enumerate(_columns):
            data = _df[_df['고장여부'] == j][_column]
            title = f'(고장={str(j)}) {_column} 분포'
            xlabel = _column
            _color = 'lightgreen' if j== 1 else 'skyblue'
            
            plt.subplot(1, 2, position)
            sns.histplot(data, bins=10, kde=True, color=_color)
            plt.title(title)
            plt.xlabel(xlabel)
            plt.ylabel('빈도수')
            plt.xlim(x_min, x_max)
            plt.ylim(0, max_count * 1.1)  # y축 범위 통일
            
            # x축의 지수 표현 제거
            #plt.ticklabel_format(axis="x", style="plain")
    
            position = position + 1
    
    plt.tight_layout()
    plt.show()

In [ ]:
df_Bike_latest_3[df_Bike_latest_3['고장여부'] == 0].head(2)

In [ ]:
df_Bike_latest_3[df_Bike_latest_3['고장여부'] == 1].head(2)

In [ ]:
df_Bike_latest_3.sort_values(by = '이전일자_1mon_이용횟수', ascending = False).head(3)

In [ ]:
# df_Bike_latest_3.columns
_columns = ['이상패턴_5min_횟수',
       '이전일자_1mon_이용거리', '이전일자_1mon_이용시간', '이전일자_1mon_이용횟수']
grb_Bike_latest_3 = df_Bike_latest_3.groupby(['고장여부'])[_columns].agg('mean')

In [ ]:
grb_Bike_latest_3.columns

In [ ]:
grb_Bike_latest_3.isnull().sum()

---
### 이용거리_비교 - 무한값(inf) 설정관련

In [ ]:
print(df_Bike_latest_3['이용거리_비교'].min())
print(df_Bike_latest_3['이용거리_비교'].max())

In [ ]:
df_Bike_latest_3[np.isinf(df_Bike_latest_3['이용거리_비교'])]['이용거리_비교']

In [ ]:
cond = df_Bike_latest_3['자전거번호'].isin(['SPB-35465', 'SPB-36015', 'SPB-37735', 'SPB-51246'])
df_Bike_latest_3 = df_Bike_latest_3[~cond]

---
### 결측치 확인 및 제거

In [ ]:
df_Bike_latest_3.isnull().mean()

In [ ]:
df_Bike_latest_3 = df_Bike_latest_3.dropna()

---
## 이상패턴_5min_횟수(고장=1, 고장=0) 시각화

In [ ]:
_columns = ['이상패턴_5min_횟수']
create_bi_subplot(check_outlier(df_Bike_latest_3[df_Bike_latest_3['이상패턴_5min_횟수'] > 5], '이상패턴_5min_횟수'), _columns)
create_bi_subplot(df_Bike_latest_3[df_Bike_latest_3['이상패턴_5min_횟수'] > 5], _columns)
create_bi_subplot(df_Bike_latest_3, _columns)

---
## 이전일자_1mon_이용거리(고장=1, 고장=0) 시각화

In [ ]:
_columns = ['이전일자_1mon_이용거리']
create_bi_subplot(check_outlier(df_Bike_latest_3[df_Bike_latest_3['이상패턴_5min_횟수'] > 5], '이상패턴_5min_횟수'), _columns)
create_bi_subplot(df_Bike_latest_3[df_Bike_latest_3['이상패턴_5min_횟수'] > 5], _columns)
create_bi_subplot(df_Bike_latest_3, _columns)

---
## 이전일자_1mon_이용시간(고장=1, 고장=0)  시각화

In [ ]:
_columns = ['이전일자_1mon_이용시간']
create_bi_subplot(check_outlier(df_Bike_latest_3[df_Bike_latest_3['이상패턴_5min_횟수'] > 5], '이상패턴_5min_횟수'), _columns)
create_bi_subplot(df_Bike_latest_3[df_Bike_latest_3['이상패턴_5min_횟수'] > 5], _columns)
create_bi_subplot(df_Bike_latest_3, _columns)

---
## 이전일자_1mon_이용횟수(고장=1, 고장=0) 시각화

In [ ]:
_columns = ['이전일자_1mon_이용횟수']
create_bi_subplot(check_outlier(df_Bike_latest_3[df_Bike_latest_3['이상패턴_5min_횟수'] > 5], '이상패턴_5min_횟수'), _columns)
create_bi_subplot(df_Bike_latest_3[df_Bike_latest_3['이상패턴_5min_횟수'] > 5], _columns)
create_bi_subplot(df_Bike_latest_3, _columns)

---
## 이용횟수_비교(고장=1, 고장=0) 시각화

In [ ]:
_columns = ['이용횟수_비교']
create_bi_subplot(check_outlier(df_Bike_latest_3[df_Bike_latest_3['이상패턴_5min_횟수'] > 5], '이상패턴_5min_횟수'), _columns)
create_bi_subplot(df_Bike_latest_3[df_Bike_latest_3['이상패턴_5min_횟수'] > 5], _columns)
create_bi_subplot(df_Bike_latest_3, _columns)

---
## 이용거리_비교(고장=1, 고장=0) 시각화

In [ ]:
_columns = ['이용거리_비교']
create_bi_subplot(check_outlier(df_Bike_latest_3[df_Bike_latest_3['이상패턴_5min_횟수'] > 5], '이상패턴_5min_횟수'), _columns)
create_bi_subplot(df_Bike_latest_3[df_Bike_latest_3['이상패턴_5min_횟수'] > 5], _columns)
create_bi_subplot(df_Bike_latest_3, _columns)

In [ ]:
df_Bike_latest_2['이상패턴_5min_횟수'].value_counts().sort_index()[:10]

In [ ]:
# 고장횟수에 따른 고장여부 비율 계산
df_2 = df_Bike_latest_2.groupby('이상패턴_5min_횟수')['고장여부'].value_counts(normalize=True).unstack().reset_index()
df_2.columns = ['이상패턴_5min_횟수', '정상(%)', '고장(%)']

result_df = df_2
# 결과 출력
result_df.head(20)


In [ ]:
# 고장횟수에 따른 고장여부 비율 계산
df_2 = df_Bike_latest_2.groupby('이상패턴_5min_횟수')['고장여부'].value_counts().unstack().reset_index()
df_2.columns = ['이상패턴_5min_횟수', '정상', '고장']

# '이상패턴_5min_횟수'의 각각의 개수 계산
df_counts = df_Bike_latest_2['이상패턴_5min_횟수'].value_counts().reset_index()
df_counts.columns = ['이상패턴_5min_횟수', '전체횟수']

# df_2와 df_counts를 병합
result_df = pd.merge(df_2, df_counts, on='이상패턴_5min_횟수', how='left').sort_values(by = '이상패턴_5min_횟수', ascending = True).head(20)

result_df['정상'] = result_df['정상'].astype(int)
result_df['고장'] = result_df['고장'].astype(int)

# 결과 출력
result_df.head(20)

---
### 시각화 - 이상패턴 <-> 고장여부

In [ ]:
# 한글 폰트 설정 (Windows에서는 'Malgun Gothic' 사용)
plt.rcParams['font.family'] = 'Malgun Gothic'
# 마이너스 기호 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False

# 결과 시각화
# result = df_Bike_latest_2[df_Bike_latest_2['이상패턴_5min_횟수'] < 30].groupby('이상패턴_5min_횟수')['고장여부'].value_counts().unstack().fillna(0)
result = df_Bike_latest_2.groupby('이상패턴_5min_횟수')['고장여부'].value_counts().unstack().fillna(0)
result.plot(kind='bar', stacked=True, figsize=(10, 6), color=['skyblue', 'salmon'])
plt.xlabel('이상패턴_5min_횟수')
plt.ylabel('횟수')

# X축과 Y축 눈금 글씨 크기 조정
plt.xticks(fontsize=10)  # X축 눈금 글씨 크기
plt.yticks(fontsize=12)  # Y축 눈금 글씨 크기

plt.title('"이상패턴_5min_횟수" 에 따른 고장여부 횟수')
plt.legend(title='고장여부')
plt.tight_layout()
plt.show()

In [ ]:
# 한글 폰트 설정 (Windows에서는 'Malgun Gothic' 사용)
plt.rcParams['font.family'] = 'Malgun Gothic'
# 마이너스 기호 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False

# 결과 시각화
result = df_Bike_latest_2.groupby('이상패턴_5min_횟수')['고장여부'].value_counts(normalize=True).unstack().fillna(0)
#result = df_Bike_latest_2.groupby('이상패턴_횟수')['고장여부'].value_counts().unstack().fillna(0)
result.plot(kind='bar', stacked=True, figsize=(10, 6), color=['skyblue', 'salmon'])
plt.xlabel('이상패턴_5min_횟수')
plt.ylabel('비율')

# X축과 Y축 눈금 글씨 크기 조정
plt.xticks(fontsize=10)  # X축 눈금 글씨 크기
plt.yticks(fontsize=12)  # Y축 눈금 글씨 크기

plt.title('"이상패턴_5min_횟수" 에 따른 고장여부 비율')
plt.legend(title='고장여부')
plt.tight_layout()
plt.show()

In [ ]:
df_Bike_latest_2.head(2)

In [ ]:
df_Bike_latest_3 = pd.get_dummies(df_Bike_latest_2, columns=['고장구분'])

In [ ]:
df_Bike_latest_3.head(3)

In [ ]:
df_Bike_latest_3.columns
df_Bike_latest_3[['고장여부',
       '이상패턴_5min_횟수', 
       '이전일자_1mon_이용거리', '이전일자_1mon_이용시간', '이전일자_1mon_이용횟수',
       '이용횟수_비교', '이용거리_비교',
       '이용횟수_비교(절대값)', '이용거리_비교(절대값)',
       '고장구분_', '고장구분_기타 ', '고장구분_단말기', '고장구분_안장', '고장구분_체인', '고장구분_타이어 ',
       '고장구분_페달']].corr()

In [ ]:
# df_Bike_latest_4 = check_outlier(df_Bike_latest_3[df_Bike_latest_3['이상패턴_5min_횟수'] > 0], '이상패턴_5min_횟수')
df_Bike_latest_4 = df_Bike_latest_3[df_Bike_latest_3['이상패턴_5min_횟수'] > 5]

df_Bike_latest_4 = df_Bike_latest_4[['고장여부',
       '이상패턴_5min_횟수',
       '이전일자_1mon_이용거리', '이전일자_1mon_이용시간', '이전일자_1mon_이용횟수',
       '이용횟수_비교', '이용거리_비교',
       '이용횟수_비교(절대값)', '이용거리_비교(절대값)',
       '고장구분_', '고장구분_기타 ', '고장구분_단말기', '고장구분_안장', '고장구분_체인', '고장구분_타이어 ',
       '고장구분_페달']].corr()

import matplotlib.pyplot as plt
import seaborn as sns

# 한글 폰트 설정 (Windows에서는 'Malgun Gothic' 사용)
plt.rcParams['font.family'] = 'Malgun Gothic'
# 마이너스 기호 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False

sns.heatmap(df_Bike_latest_4.corr(),
            annot = True,                     # 상관계수 표시
            annot_kws={"size": 8},
            fmt = '.1f',                      # 상관계수 소수점 자리
            cmap = 'coolwarm',                # 컬러맵 색상 팔레트
            vmax = 1.0,                       # 상관계수 최댓값
            vmin = -1,                       # 상관계수 최소값
            linecolor = 'white',              # 셀 테두리 색상
            linewidths = .005)                 # 셀 간격
sns.set(rc={'figure.figsize' : (9, 9)})       # 그래프 그리기

In [ ]:
df_Bike_latest_3_2.describe()

In [ ]:
outliers.head(2)

In [ ]:
outliers['고장여부'].value_counts()

---
## 로지스틱회귀 모델링

In [ ]:
# outliers_3 = check_outlier(df_Bike_latest_3[df_Bike_latest_3['이상패턴_5min_횟수'] > 0], '이상패턴_5min_횟수')
# outliers_3 = outliers[outliers['이상패턴_5min_횟수'] > 1]
#outliers_3 = check_outlier(df_Bike_latest_3[df_Bike_latest_3['이상패턴_5min_횟수'] > 1], '이상패턴_5min_횟수')


# outliers_3 = outliers[outliers['이상패턴_5min_횟수'] > 0]
# - 테스트 세트 정확도: 0.7186932849364791
# outliers_3 = outliers[outliers['이상패턴_5min_횟수'] > 1]
# - 테스트 세트 정확도: 0.7407467797162889
# outliers_3 = outliers[outliers['이상패턴_5min_횟수'] > 2]
# - 테스트 세트 정확도: 0.758641600970285
# outliers_3 = outliers[outliers['이상패턴_5min_횟수'] > 3]
# - 테스트 세트 정확도: 0.7893236589790101
# outliers_3 = outliers[outliers['이상패턴_5min_횟수'] > 4]
# - 테스트 세트 정확도: 0.8163196776359973
outliers_3 = outliers[outliers['이상패턴_5min_횟수'] > 5]
# - 테스트 세트 정확도: 0.8446771378708552

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, f1_score, log_loss

# 한글 폰트 설정 (Windows에서는 'Malgun Gothic' 사용)
plt.rcParams['font.family'] = 'Malgun Gothic'
# 마이너스 기호 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt

# '이상패턴_5min_횟수', '이전일자_1mon_이용횟수', '이용횟수_비교', '이용거리_비교'
# ============================================================================
# _columns = ['이상패턴_5min_횟수']                                                           # 테스트 세트 정확도: 0.8259162303664922
# _columns = ['이상패턴_5min_횟수', '이용횟수_비교']                                           # 테스트 세트 정확도: 0.8259162303664922
# _columns = ['이상패턴_5min_횟수', '이전일자_1mon_이용횟수']                                  # 테스트 세트 정확도: 0.8259162303664922
# _columns = ['이상패턴_5min_횟수', '이용거리_비교']                                           # 테스트 세트 정확도: 0.8259162303664922
# _columns = ['이용횟수_비교']                                                                # 테스트 세트 정확도: 0.8259162303664922

# _columns = ['이용거리_비교']                                                                # 테스트 세트 정확도: 0.8263525305410122
# _columns = ['이용횟수_비교', '이용거리_비교']                                                # 테스트 세트 정확도: 0.8263525305410122
# _columns = ['이상패턴_5min_횟수', '이용횟수_비교', '이용거리_비교']                           # 테스트 세트 정확도: 0.8294066317626527
# _columns = ['이상패턴_5min_횟수', '이전일자_1mon_이용횟수', '이용거리_비교']                  # 테스트 세트 정확도: 0.8328970331588132
# _columns = ['이상패턴_5min_횟수', '이전일자_1mon_이용횟수', '이용횟수_비교', '이용거리_비교']  # 테스트 세트 정확도: 0.8442408376963351
_columns = ['이상패턴_5min_횟수', '이전일자_1mon_이용횟수', '이용횟수_비교']                  # 테스트 세트 정확도: 0.8446771378708552


X = np.array(outliers_3[_columns]).reshape(-1, len(_columns))
y = np.array(outliers_3['고장여부'])

# 데이터셋을 학습 세트와 테스트 세트로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=444, stratify=y)
print(X_train.shape[0], X_test.shape[0], y_train.shape[0], y_test.shape[0])

# 로지스틱 회귀 모델 생성 및 학습
model = LogisticRegression()
model.fit(X_train, y_train)

# 테스트 세트에 대한 예측
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

# # 결과 시각화
# plt.scatter(X_test[:, 1], y_test, color='blue', label='Actual')
# plt.scatter(X_test[:, 1], y_pred, color='red', label='Predicted')
# plt.xlabel('"이상대여패턴"')
# plt.ylabel('고장여부')
# plt.title('"이상대여패턴" 에 따른 고장여부 예측')
# plt.legend()
# plt.show()

# 모델 평가
print('테스트 세트 정확도 : ', model.score(X_test, y_test))
print('confusion_matrix : \n', confusion_matrix(y_test, y_pred))
print('classification_report : \n', classification_report(y_test, y_pred))
print("Accuracy : ", accuracy_score(y_test, y_pred))
print("F1 Score : ", f1_score(y_test, y_pred))
print("Log Loss : ", log_loss(y_test, y_prob))


In [ ]:
# 회귀 계수 출력
print('회귀 계수:', model.coef_)
print('절편:', model.intercept_)

In [ ]:
from sklearn.metrics import roc_curve, auc

fpr, tpr, _ = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

### [ 이상패턴 횟수 == 0 ] 것들의 고장여부 비율

In [ ]:
counts = df_Bike_latest_2[df_Bike_latest_2['이상패턴_5min_횟수'] > 5]['고장여부'].value_counts()
total = counts.sum()   # 총 데이터 개수
percentage = counts / total * 100   # 비율 계산
print(f'total : {total}')
print(f'counts : {counts}')
print(f'percentage : {percentage}')